In [1]:
import sys
import inspect
from jw_utils import jw_draw_tree as jdt
from ete3 import ncbi_taxonomy
from ete3 import Tree
ncbi_tax = ncbi_taxonomy.NCBITaxa()
import json
from jw_utils import ncbi_datasets_fxs as nfx
from orthofinder_utils import dash_ortho_parser as dop
from Bio import Phylo
from plotly import graph_objects as go
from jw_utils import jw_ncbi_taxonomy as jnt
import os
from jw_utils import plotly_preferences as pprefs
pprefs.__file__

'/Users/jonwinkelman/python-modules/jw_utils/plotly_preferences.py'

In [2]:
rgb_to_hex = {'rgba(132,137,145,1)':'#848991',
 'rgb(0,208,132)':'#00d084',
 'rgb(0,122,255)':'#007aff',
 'rgb(171,184,195)':'#abb8c3',
 'rgb(255,105,0)':'#ff6900',
 'rgb(252,185,0)':'#fcb900',
 'rgb(123,220,181)':'#7bdcb5',
 'rgb(142,209,252)':'#8ed1fc',
 'rgb(6,147,227)':'#0693e3',
 'rgb(155,81,224)':'#9b51e0',
 'rgb(207,46,46)':'#cf2e2e',
 'rgb(247,141,167)':'#f78da7'}

hex_colors = list(rgb_to_hex.values())

In [73]:
HOGs = {
    'crc':'N0.HOG0003265', 'cbrA':'N0.HOG0003702', 'pelA':'N0.HOG0007706','pelB':'N0.HOG0007705','pelC':'N0.HOG0007650','cupB1':'N0.HOG0005381',
    'cupB2':'N0.HOG0000906','cupB3':'N0.HOG0000906','cupC1':'N0.HOG0005710','cupC2':'N0.HOG0000907','cupC3':'N0.HOG0001149',
    }
HOGs_truncated = {
    'cbrA':'N0.HOG0003702', 'crc':'N0.HOG0003265', 'pelA':'N0.HOG0007706','cupB1':'N0.HOG0005381','cupC2':'N0.HOG0000907',
     }
        


In [4]:

name_list, count_dict =  make_itol_count_dict(tree, HOGs_to_name_d, dop_obj)

NameError: name 'make_itol_count_dict' is not defined

In [120]:
itol_outfile_path = './itol_tree/multibar_dataset_trunc.txt'
binary_count_dict = make_count_dict_binary(count_dict)

In [124]:
make_multi_itol_bargraph_dataset(itol_outfile_path, count_dict=binary_count_dict, name_list=name_list)  

In [170]:
def relabel_itol_treeleafs(tree, relabel_dict, outfile_path):
    """"""
    import warnings
    tree_leafnames = [cl.name for cl in tree.get_terminals()]
    if len(tree_leafnames) != len(relabel_dict.keys()):
        warnings.warn('The number of tree leafs and the number of dict key names to be replaced are not equal')
    with open(outfile_path, 'w') as f:
        f.write('LABELS\n')
        f.write('SEPARATOR COMMA\n')
        f.write('DATA\n')
        for old_name, new_name in relabel_dict.items():
            f.write(f'{old_name},{new_name}\n')
    



In [180]:
relabel_dict = {}
for acc, name in dop_obj.accession_to_name.items():
    name = name.replace('[','').replace(']','')
    name_list =  name.split(' ')
    genus = name_list[0][:50]+'.'
    species = name_list[1][:7]+'.'
    strain = ' '.join(name_list[2:])[:10]
    strain = name_list[-1][:10]
    name = f'{genus} {species} {strain}' 
    relabel_dict[acc]=name
print(len(relabel_dict))
    
outfile_path = './itol_tree/rename_dataset.txt'
relabel_itol_treeleafs(tree, relabel_dict, outfile_path)

234


In [192]:
import shutil
dest = './Proteomes'
parent_fp = './data/ncbi_dataset/data/'
for folder in [f for f in os.listdir(parent_fp) if f.startswith('GC')]:
    proteome_fp = os.path.join(parent_fp,folder, 'sequence_report.jsonl')
    if os.path.exists(proteome_fp):
        n_dest_fp = os.path.join(dest, folder)
        shutil.move(proteome_fp, n_dest_fp)

In [49]:
def make_simple_itol_bargraph_dataset(template_path, file_path, data_dict):
    """
    Add data to simple bargraph template for itol dataset adn write to file.

    Parameters
    ----------
    template_path : str
        path to template file from itol.
    file_path : str
        path for new file that is to be created
    data_dict :dict
        {terminal node name: count}

    """


    template_lines = []
    with open(template_path, 'r') as f:
        for line in f:
            if line[-1] != '\n':
                line = line+'\n'
            template_lines.append(line)
            
    with open(file_path, 'w') as f:
        for line in template_lines:
            f.write(line)
        for name, count in data_dict.items():
            f.write(f'{name},{count}\n')
            
crc_data = list(copynumber_bargraph_data_dict([HOGs['crc']], dop_obj, id_type='accession').values())[0]
template_path = './itol_tree/dataset_multibar_template.txt'
file_path = './itol_tree/crc_simplebar_dataset.txt'
make_simple_itol_bargraph_dataset(template_path, file_path, crc_data)